In [1]:
import pandas as pd
import geopandas as gpd
import seaborn as sns
import numpy as np
from sklearn import preprocessing
from sklearn.pipeline import Pipeline
from sklearn.datasets import load_iris
from sklearn.svm import LinearSVC
from sklearn.naive_bayes import GaussianNB
from sklearn.metrics import accuracy_score
from sklearn.pipeline import Pipeline
from sklearn.model_selection import GridSearchCV
from sklearn.metrics import precision_score
import datetime

/Users/marielwiechers/my_env/lib/python3.9/site-packages/geopandas/_compat.py:106: UserWarning: The Shapely GEOS version (3.8.0-CAPI-1.13.1 ) is incompatible with the GEOS version PyGEOS was compiled with (3.9.0-CAPI-1.16.2). Conversions between both will be slow.
  warnings.warn(


In [8]:
%run ml_helpers.ipynb
%run ml.ipynb

         Unnamed: 0  Beat  Year  Month   Watch  PRCP  SNOW  TMAX  TMIN  \
1311550     1311550  1031  2019     12   Third  0.10   0.0    43    33   
1292209     1292209   313  2019     11  Second  0.00   0.0    45    33   
1252286     1252286   431  2019      9   First  0.05   0.0    67    62   
1071746     1071746   412  2019      1  Second  0.00   0.1    36    27   
1249411     1249411  2535  2019      9   First  0.00   0.0    70    62   
...             ...   ...   ...    ...     ...   ...   ...   ...   ...   
1214549     1214549  1221  2019      7   Third  0.81   0.0    80    71   
1138628     1138628  1531  2019      4   Third  0.00   0.0    54    37   
1090948     1090948  1532  2019      2  Second  0.02   0.4    21     2   
1107863     1107863  2521  2019      2   Third  0.00   0.0    25    18   
1309470     1309470  1935  2019     11   Third  0.00   0.0    39    32   

         count_l_stops  ...  count_restaurants  count_bars  count_daycares  \
1311550            0.0  ...      

Finished one-hot encoding...
Finished one-hot encoding...
Finished standardizing...


In [3]:
data = pd.read_csv("../intermediate_data/high_crime_labeled.csv")
data = data.drop(columns=["beat", "beat_num", "Crimes", "Arrest", "Domestic", "Serious"])
data = convert_to_categorical(data, ["district", "sector",
                                    "Month",
                                     "Watch", "Beat"])
data.drop(columns = ["Unnamed: 0"], inplace=True)

In [4]:
data.head()

,district,sector,Year,Month,Watch,Beat,TMAX,TMIN,PRCP,SNOW,...,count_restaurants,count_bars,count_daycares,count_entertainment,count_businesses,road_distance_ft,TOTAL POPULATION,dist_to_police,dist_to_hospital,high_crime_geog_lag
0,17,1,2015,1,Third,1713,32.469987,20.138549,1.55,11.3,...,63.0,9.0,6.0,1.0,92.0,169276.405792,13283.675264,5454.06889,3258.006066,0.155378
1,17,1,2015,1,Second,1713,32.469987,20.138549,1.55,11.3,...,63.0,9.0,6.0,1.0,92.0,169276.405792,13283.675264,5454.06889,3258.006066,0.155378
2,17,1,2015,1,First,1713,32.469987,20.138549,1.55,11.3,...,63.0,9.0,6.0,1.0,92.0,169276.405792,13283.675264,5454.06889,3258.006066,0.155378
3,17,1,2015,2,Second,1713,26.455588,10.647633,1.87,28.3,...,63.0,9.0,6.0,1.0,92.0,169276.405792,13283.675264,5454.06889,3258.006066,0.155378
4,17,1,2015,2,Third,1713,26.455588,10.647633,1.87,28.3,...,63.0,9.0,6.0,1.0,92.0,169276.405792,13283.675264,5454.06889,3258.006066,0.155378


In [5]:
models = {
    'GaussianNB': GaussianNB(),
    'LinearSVC': LinearSVC()    
}

NBgrid = {'GaussianNB': [{'priors': None}] 
                         #for x in ('None')]
         }

SVCgrid = {'LinearSVC': [{'C': x, 'random_state': 0} for x in (0.01, 0.1, 1, 10, 100)]}

In [6]:
data_list = prep_data(data, "high_crime",
                                        2, "Year", ["district", "sector", 
                                                    "Month", 
                                                    "Beat", "Watch"])

      district sector  Year Month   Watch  Beat       TMAX       TMIN  PRCP  \
29557       17      1  2018     1   First  1713  33.201426  18.884223  1.12   
29558       17      1  2018     1  Second  1713  33.201426  18.884223  1.12   
29559       17      1  2018     1   Third  1713  33.201426  18.884223  1.12   
29560       17      1  2018     2   Third  1713  39.791635  22.439919  5.47   
29561       17      1  2018     2   First  1713  39.791635  22.439919  5.47   
...        ...    ...   ...   ...     ...   ...        ...        ...   ...   
49256        3      1  2019    11  Second   312  43.761013  29.648049  1.70   
49257        3      1  2019    11   Third   312  43.761013  29.648049  1.70   
49258        3      1  2019    12  Second   312  42.647269  27.854899  1.29   
49259        3      1  2019    12   First   312  42.647269  27.854899  1.29   
49260        3      1  2019    12   Third   312  42.647269  27.854899  1.29   

       SNOW  ... count_restaurants  count_bars  cou

/Users/marielwiechers/my_env/lib/python3.9/site-packages/pandas/core/series.py:4463: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  return super().fillna(
<ipython-input-2-bc70820a9afc>:12: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df["Year"]=df["Year"].astype("category")
/Users/marielwiechers/my_env/lib/python3.9/site-packages/pandas/core/indexing.py:1738: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pyda

Finished normalizing training data
Finished normalizing test data...
Finished one-hot encoding...
Finished one-hot encoding...
Finished standardizing...
Working on: [2016 2017]
Have accessed train and test df...
Finished filling NAs with mean...
On to normalizing continuous...
Finished normalizing training data
Finished normalizing test data...
Finished one-hot encoding...
Finished one-hot encoding...
Finished standardizing...
Working on: [2015 2016]
Have accessed train and test df...
Finished filling NAs with mean...
On to normalizing continuous...


/Users/marielwiechers/my_env/lib/python3.9/site-packages/pandas/core/series.py:4463: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  return super().fillna(
<ipython-input-2-bc70820a9afc>:12: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df["Year"]=df["Year"].astype("category")
/Users/marielwiechers/my_env/lib/python3.9/site-packages/pandas/core/indexing.py:1738: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pyda

Finished normalizing training data
Finished normalizing test data...
Finished one-hot encoding...
Finished one-hot encoding...
Finished standardizing...


In [9]:
nb_results = average_grid_searches(GaussianNB(), 'GaussianNB', NBgrid, data_list, "high_crime", 2020)

test year is: 2020
[2016, 2017, 2018, 2019]


In [17]:
nb_results

,parameters_2020,precision_2020,parameters_2019,precision_2019,parameters_2018,precision_2018,parameters_2017,precision_2017,mean
0,{'priors': None},0.35951,{'priors': None},0.38322,{'priors': None},0.363594,{'priors': None},0.355835,0.36554


In [14]:
nb_results.to_csv("../final_data/NB.csv")

In [11]:
svc_results = average_grid_searches(LinearSVC(), 'LinearSVC', SVCgrid, data_list, "high_crime", 2020)

test year is: 2020
[2016, 2017, 2018, 2019]


In [16]:
svc_results

,parameters_2020,precision_2020,parameters_2019,precision_2019,parameters_2018,precision_2018,parameters_2017,precision_2017,mean
0,"{'C': 1, 'random_state': 0}",0.257858,"{'C': 1, 'random_state': 0}",0.751169,"{'C': 1, 'random_state': 0}",1.000000,"{'C': 1, 'random_state': 0}",0.927184,0.734053
0,"{'C': 10, 'random_state': 0}",0.257858,"{'C': 10, 'random_state': 0}",0.751169,"{'C': 10, 'random_state': 0}",1.000000,"{'C': 10, 'random_state': 0}",0.927184,0.734053
0,"{'C': 100, 'random_state': 0}",0.257858,"{'C': 100, 'random_state': 0}",0.751169,"{'C': 100, 'random_state': 0}",1.000000,"{'C': 100, 'random_state': 0}",0.927184,0.734053
0,"{'C': 0.1, 'random_state': 0}",0.257806,"{'C': 0.1, 'random_state': 0}",0.751169,"{'C': 0.1, 'random_state': 0}",1.000000,"{'C': 0.1, 'random_state': 0}",0.843496,0.713118
0,"{'C': 0.01, 'random_state': 0}",0.258441,"{'C': 0.01, 'random_state': 0}",0.265156,"{'C': 0.01, 'random_state': 0}",0.282265,"{'C': 0.01, 'random_state': 0}",0.738462,0.386081


In [15]:
svc_results.to_csv("../final_data/SVC.csv")